In [69]:
import pandas as pd
import streamlit_authenticator as stauth
import sqlite3
conn_sqlite = sqlite3.connect('credentials.db')
c = conn_sqlite.cursor()
import hashlib
import string
import random

In [70]:
data = pd.read_excel('pegawai110.xlsx', dtype={'NIP':'str','KD':'str'},usecols=['NIP','Nama','Kantor','KD'])

In [71]:
def generate_random_string(length):
    random.seed=10
    letters = string.ascii_lowercase + string.ascii_uppercase + string.digits
    return ''.join(random.choice(letters) for _ in range(length))

In [72]:
for index,row in data.iterrows():    
    data['password'] = data['NIP'].apply(lambda x: x+'@'+generate_random_string(4))

In [73]:
data

,Nama,NIP,Kantor,KD,password
0,MUHAMMAD ISMIRANSYAH M. ZAIN,060077930,Kanwil DJP Jakarta Timur,110,060077930@FcS7
1,SUGENG SATOTO,060094070,Kanwil DJP Jakarta Timur,110,060094070@TmoT
2,DESSY EKA PUTRI,060087065,Kanwil DJP Jakarta Timur,110,060087065@dYcU
3,SANDRA BUANA,060083161,Kanwil DJP Jakarta Timur,110,060083161@rgGi
4,ARDHIE PERMADI,060083105,Kanwil DJP Jakarta Timur,110,060083105@Fum4
...,...,...,...,...,...
1082,NASTRIA PRATAMA,930302289,KPP Madya Dua Jakarta Timur,097,930302289@NF2v
1083,ADRY NOVIANTO,860013856,KPP Madya Dua Jakarta Timur,097,860013856@R1PX
1084,HERU IRFANTO,808320132,KPP Madya Dua Jakarta Timur,097,808320132@IqxB
1085,MIFTAKHUL PRATAMA PUTRA,810600567,KPP Madya Dua Jakarta Timur,097,810600567@pZUh


In [82]:
def make_hashes(password):
	return stauth.Hasher(password).generate()
def check_hashes(password,hashed_text):
	if make_hashes(password) == hashed_text:
		return hashed_text
	return False
def add_userdata(username,nama,password):
	c.execute('INSERT INTO user(username,name,password) VALUES (?,?,?)',(username,nama,password))
	conn_sqlite.commit()

In [79]:
c.execute('CREATE TABLE IF NOT EXISTS user(username TEXT,name TEXT,password TEXT)')
conn_sqlite.commit()

In [76]:
usernames = data['NIP'].tolist()
names =data['Nama'].tolist()
password = data['password'].tolist()
hashed_password = stauth.Hasher(password).generate()

In [83]:
for (username, name, hash_password) in zip(usernames, names, hashed_password):
    add_userdata(username, name, hash_password)

In [84]:
# data.to_excel('login.xlsx',index=False)

In [62]:
for index,row in data.iterrows():
    add_userdata(row['NIP'],row['Nama'],make_hashes(row['password']),row['KD'],row['Kantor'])

In [15]:
pass_hashed = stauth.Hasher(data['password'].tolist()).generate()

In [17]:
secrets = pd.DataFrame(pass_hashed, index=data.NIP.tolist(),columns=['PASS_HASHED'])

In [20]:
credential = pd.merge(data, secrets, left_on='NIP', right_index=True, how='inner')

In [21]:
credential

,Nama,NIP,Kantor,KD,password,PASS_HASHED
0,MUHAMMAD ISMIRANSYAH M. ZAIN,060077930,Kanwil DJP Jakarta Timur,110,060077930@Jaktim110,$2b$12$TXqPWPLGgXDy0hlQHwtBCu1D578eRjR9pKhWDAk...
1,SUGENG SATOTO,060094070,Kanwil DJP Jakarta Timur,110,060094070@Jaktim110,$2b$12$M9q6I6zd4bOPVexnHSvSU.g3ROlRPVNsFCyZRNp...
2,DESSY EKA PUTRI,060087065,Kanwil DJP Jakarta Timur,110,060087065@Jaktim110,$2b$12$rmV0tU0eViezIdEzA259dOuwYEJVwnPQJkVwxjd...
3,SANDRA BUANA,060083161,Kanwil DJP Jakarta Timur,110,060083161@Jaktim110,$2b$12$D/.gereEl4T..Y72GI4rm.rTRs9a6xRHcGiPhnf...
4,ARDHIE PERMADI,060083105,Kanwil DJP Jakarta Timur,110,060083105@Jaktim110,$2b$12$ryFdEuziF0EzMziPyI3VC.gyE5i.BS3XuDDdSKC...
...,...,...,...,...,...,...
1082,NASTRIA PRATAMA,930302289,KPP Madya Dua Jakarta Timur,097,930302289@Jaktim110,$2b$12$YGzPzHCcmbOBQwyM6BdPreJc1e2cIS1evOgUEha...
1083,ADRY NOVIANTO,860013856,KPP Madya Dua Jakarta Timur,097,860013856@Jaktim110,$2b$12$wdKWPEa1FYwCIwXR1al5sefSz/yPO8vpM6DIUuw...
1084,HERU IRFANTO,808320132,KPP Madya Dua Jakarta Timur,097,808320132@Jaktim110,$2b$12$OkFmwV4XvkIM5dO4wtoe7eAmwXiCilFuk4/pu8t...
1085,MIFTAKHUL PRATAMA PUTRA,810600567,KPP Madya Dua Jakarta Timur,097,810600567@Jaktim110,$2b$12$uqN/fCc5En/Jz0nAEgvUTeX0VCbOgaVf16Qq8AX...


In [23]:
db.sql('CREATE TABLE credential AS SELECT * FROM credential')

In [6]:
db.close()

In [68]:
c.execute('DROP TABLE user')
conn_sqlite.commit()

In [64]:
from yaml.loader import SafeLoader
import yaml
with open(".streamlit/login.yaml") as file:
    config = yaml.load(file, Loader=SafeLoader)

In [85]:
cred =config['credentials']

In [93]:
cred

{'usernames': {'060098992': {'email': None,
   'name': 'HERMAWAN',
   'password': '$2b$12$q0JaQ8JKzS4DxMsS8Y2OJOyklNImhwBHZwhp0CLre2sA.nI.lH53C'},
  '060077930': {'email': None,
   'name': 'MUHAMMAD ISMIRANSYAH M. ZAIN',
   'password': '$2b$12$2iOgkgdCRpBXziJ6qOuApu4ERfclryQUeVaODzWmZoO7i0fjQRo1q'},
  '060087065': {'email': None,
   'name': 'DESSY EKA PUTRI',
   'password': '$2b$12$YiVsALwTjDQyTVVKZqlgUOt.dLArM.hlEjckznjqrrJS47rOREnjS'},
  '910222266': {'email': None,
   'name': 'EMANUEL EKO DARMAWAN',
   'password': '$2b$12$0H5.Yrhv.c1TCzf3TfkE6ecjI2hc.UWqNGQSHAYHObkgLOwF35.jO'},
  '958633540': {'email': None,
   'name': 'WIRAWAN THO`AT ARUMDHANI',
   'password': '$2b$12$K23Mk7N2DBHld4bVIorkeeiascmHUBNCKvQSPu103fw5ocygiqbee'},
  '060105487': {'email': None,
   'name': 'HAWELL JECKSON',
   'password': '$2b$12$y0eo90dIyUApdhr79vqTse8BStnf8C7djiIb06vHoaBKUGiO8AfBe'},
  '953800975': {'email': None,
   'name': 'DEWI MARGARET LAMRIA PANGARIBUAN',
   'password': '$2b$12$CZIe1CHvSduqoHPIguwqN

In [89]:
cred['usernames']['060098992']

{'email': None,
 'name': 'HERMAWAN',
 'password': '$2b$12$q0JaQ8JKzS4DxMsS8Y2OJOyklNImhwBHZwhp0CLre2sA.nI.lH53C'}